# Introduction
In this short notebook, we transform the data about covid (add some information) in order to send it to Google Data Studio. 

In [1]:
import numpy as np
import pandas as pd


In [158]:
# the data has been downloaded from here: https://data.europa.eu/euodp/en/data/dataset/covid-19-coronavirus-data
df_covid = pd.read_csv("data/covid.csv")
df_covid.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,22/09/2020,22,9,2020,30,3,Afghanistan,AF,AFG,38041757.0,Asia,1.524640
1,21/09/2020,21,9,2020,0,0,Afghanistan,AF,AFG,38041757.0,Asia,1.698134
2,20/09/2020,20,9,2020,125,4,Afghanistan,AF,AFG,38041757.0,Asia,1.892657
3,19/09/2020,19,9,2020,47,1,Afghanistan,AF,AFG,38041757.0,Asia,1.616645
4,18/09/2020,18,9,2020,0,0,Afghanistan,AF,AFG,38041757.0,Asia,1.535155


# Data transformation and enrichment

In [159]:
# select and rename some features for better readability
df_covid = df_covid.rename(index=str, columns={
    "dateRep": "date", 
    "popData2019": "population",
    "countriesAndTerritories": "country_name",
    "geoId": "country_id",
})[[
    "date",
    "population",
    "country_name",
    "country_id",
    "cases",
    "deaths",
    "day",
    "month",
    "year",
]]

df_covid.head()

,date,population,country_name,country_id,cases,deaths,day,month,year
0,22/09/2020,38041757.0,Afghanistan,AF,30,3,22,9,2020
1,21/09/2020,38041757.0,Afghanistan,AF,0,0,21,9,2020
2,20/09/2020,38041757.0,Afghanistan,AF,125,4,20,9,2020
3,19/09/2020,38041757.0,Afghanistan,AF,47,1,19,9,2020
4,18/09/2020,38041757.0,Afghanistan,AF,0,0,18,9,2020


In [160]:
# casting of date
df_covid["date_measure"] = pd.to_datetime(
    df_covid["year"]*10000+df_covid["month"]*100+df_covid["day"],format='%Y%m%d'
)
df_covid.head()

,date,population,country_name,country_id,cases,deaths,day,month,year,date_measure
0,22/09/2020,38041757.0,Afghanistan,AF,30,3,22,9,2020,2020-09-22
1,21/09/2020,38041757.0,Afghanistan,AF,0,0,21,9,2020,2020-09-21
2,20/09/2020,38041757.0,Afghanistan,AF,125,4,20,9,2020,2020-09-20
3,19/09/2020,38041757.0,Afghanistan,AF,47,1,19,9,2020,2020-09-19
4,18/09/2020,38041757.0,Afghanistan,AF,0,0,18,9,2020,2020-09-18


In [161]:
# compute the cumulative cases and deaths
df_covid_cum = df_covid.groupby(
    ['country_name','date_measure']
).sum().groupby('country_name').cumsum()[["cases","deaths"]]
df_covid_cum = df_covid_cum.reset_index()
df_covid_cum = df_covid_cum.rename(index=str, columns={
    "cases": "cases_cumulative", 
    "deaths": "deaths_cumulative",
})
df_covid_cum.head()

,country_name,date_measure,cases_cumulative,deaths_cumulative
0,Afghanistan,2019-12-31,0,0
1,Afghanistan,2020-01-01,0,0
2,Afghanistan,2020-01-02,0,0
3,Afghanistan,2020-01-03,0,0
4,Afghanistan,2020-01-04,0,0


In [162]:
# simple sort
df_covid = pd.merge(df_covid, df_covid_cum, how='inner', on=["date_measure","country_name"])
df_covid = df_covid.sort_values(['date_measure', 'country_name'], ascending=[1, 1])
df_covid.head()

,date,population,country_name,country_id,cases,deaths,day,month,year,date_measure,cases_cumulative,deaths_cumulative
256,31/12/2019,38041757.0,Afghanistan,AF,0,0,31,12,2019,2019-12-31,0,0
716,31/12/2019,43053054.0,Algeria,DZ,0,0,31,12,2019,2019-12-31,0,0
1919,31/12/2019,2957728.0,Armenia,AM,0,0,31,12,2019,2019-12-31,0,0
2370,31/12/2019,25203200.0,Australia,AU,0,0,31,12,2019,2019-12-31,0,0
2637,31/12/2019,8858775.0,Austria,AT,0,0,31,12,2019,2019-12-31,0,0


In [163]:
# compute the proportion of cases and deaths to the each population size.
df_covid["cases_in_1M"] = df_covid.apply(
    lambda row: (1000000 * row["cases_cumulative"]) / row["population"], axis=1)
df_covid["deaths_in_1M"] = df_covid.apply(
    lambda row: (1000000 * row["deaths_cumulative"]) / row["population"], axis=1)
df_covid[df_covid["country_name"]=="France"].tail()

,date,population,country_name,country_id,cases,deaths,day,month,year,date_measure,cases_cumulative,deaths_cumulative,cases_in_1M,deaths_in_1M
14641,18/09/2020,67012883.0,France,FR,10593,50,18,9,2020,2020-09-18,415481,31095,6200.016794,464.015255
14640,19/09/2020,67012883.0,France,FR,13215,154,19,9,2020,2020-09-19,428696,31249,6397.217681,466.313321
14639,20/09/2020,67012883.0,France,FR,13498,25,20,9,2020,2020-09-20,442194,31274,6598.641637,466.686383
14638,21/09/2020,67012883.0,France,FR,11569,11,21,9,2020,2020-09-21,453763,31285,6771.280084,466.850531
14637,22/09/2020,67012883.0,France,FR,4298,53,22,9,2020,2020-09-22,458061,31338,6835.417005,467.641424


In [164]:
# rename the columns so that they are Data Studio friendly.
df_covid = df_covid[[
    "date_measure", 
    "country_name", 
    "country_id", 
    "cases",
    "deaths",
    "population",
    "cases_cumulative",
    "deaths_cumulative",
    "cases_in_1M",
    "deaths_in_1M",
]]

df_covid = df_covid.rename(index=str, columns={
    "date_measure": "Date of measure", 
    "country_name": "Country name", 
    "country_id": "Country id", 
    "cases": "Cases",
    "deaths": "Deaths",
    "population": "Population size",
    "cases_cumulative": "Cumulative cases",
    "deaths_cumulative": "Cumulative deaths",
    "cases_in_1M": "Cases to population",
    "deaths_in_1M": "Deaths to population",
})
df_covid.head()

,Date of measure,Country name,Country id,Cases,Deaths,Population size,Cumulative cases,Cumulative deaths,Cases to population,Deaths to population
256,2019-12-31,Afghanistan,AF,0,0,38041757.0,0,0,0.0,0.0
716,2019-12-31,Algeria,DZ,0,0,43053054.0,0,0,0.0,0.0
1919,2019-12-31,Armenia,AM,0,0,2957728.0,0,0,0.0,0.0
2370,2019-12-31,Australia,AU,0,0,25203200.0,0,0,0.0,0.0
2637,2019-12-31,Austria,AT,0,0,8858775.0,0,0,0.0,0.0


In [165]:
df_covid.to_csv("data/output/covid_parsed_05.csv", index=False)

# Additional analysis

In [166]:
# are there rows where the cases are negative?
df_covid[df_covid["Cases"]<0]

,Date of measure,Country name,Country id,Cases,Deaths,Population size,Cumulative cases,Cumulative deaths,Cases to population,Deaths to population
7864,2020-03-10,Cases_on_an_international_conveyance_Japan,JPG11668,-9,1,NaN,696,7,NaN,NaN
37989,2020-04-19,Spain,ES,-713,410,46937060.0,193252,20453,4117.258303,435.753752
24594,2020-04-29,Lithuania,LT,-105,3,2794184.0,1344,44,480.999104,15.746994
32981,2020-05-03,Portugal,PT,-161,16,10276617.0,25190,1023,2451.195758,99.546378
12215,2020-05-07,Ecuador,EC,-2461,49,17373657.0,29420,1618,1693.368299,93.129501
12213,2020-05-09,Ecuador,EC,-1480,50,17373657.0,28818,1704,1658.718139,98.079523
34899,2020-05-11,San_Marino,SM,-9,0,34453.0,628,41,18227.730531,1190.026993
12210,2020-05-12,Ecuador,EC,-50,18,17373657.0,29509,2145,1698.490997,123.462780
4568,2020-05-20,Benin,BJ,-209,0,11801151.0,130,2,11.015875,0.169475
41397,2020-05-21,Uganda,UG,-115,0,44269587.0,145,0,3.275386,0.000000


In [167]:
# sort by last death proportion to the population size (descending)
df_covid_temp_gp = df_covid.groupby(['Country name']).max()[["Deaths to population"]]
df_covid_temp_gp = df_covid_temp_gp.reset_index()
df_covid_temp_gp = df_covid_temp_gp.sort_values(['Deaths to population'], ascending=[0])
df_covid_temp_gp.head(10)

,Country name,Deaths to population
164,San_Marino,1219.052042
152,Peru,968.119124
18,Belgium,868.576972
3,Andorra,695.748060
23,Bolivia,664.807799
179,Spain,653.279093
27,Brazil,650.425552
41,Chile,648.901292
58,Ecuador,638.610512
198,United_Kingdom,627.003913


In [168]:
df_country_pop = df_covid[["Country name","Country id","Population size"]].drop_duplicates()
df_country_pop.to_csv("data/output/country_population.csv", index=False)

In [169]:
zipped = zip(df_country_pop["Country id"], df_country_pop["Population size"]) 
print(zipped)
country_pop_dict = dict(zipped)
print(country_pop_dict)

{'AF': 38041757.0, 'DZ': 43053054.0, 'AM': 2957728.0, 'AU': 25203200.0, 'AT': 8858775.0, 'AZ': 10047719.0, 'BH': 1641164.0, 'BY': 9452409.0, 'BE': 11455519.0, 'BR': 211049519.0, 'KH': 16486542.0, 'CA': 37411038.0, 'JPG11668': nan, 'CN': 1433783692.0, 'HR': 4076246.0, 'CZ': 10649800.0, 'DK': 5806081.0, 'DO': 10738957.0, 'EC': 17373657.0, 'EG': 100388076.0, 'EE': 1324820.0, 'FI': 5517919.0, 'FR': 67012883.0, 'GE': 3996762.0, 'DE': 83019213.0, 'EL': 10724599.0, 'IS': 356991.0, 'IN': 1366417756.0, 'ID': 270625567.0, 'IR': 82913893.0, 'IQ': 39309789.0, 'IE': 4904240.0, 'IL': 8519373.0, 'IT': 60359546.0, 'JP': 126860299.0, 'KW': 4207077.0, 'LB': 6855709.0, 'LT': 2794184.0, 'LU': 613894.0, 'MY': 31949789.0, 'MX': 127575529.0, 'MC': 33085.0, 'NP': 28608715.0, 'NL': 17282163.0, 'NZ': 4783062.0, 'NG': 200963603.0, 'MK': 2077132.0, 'NO': 5328212.0, 'OM': 4974992.0, 'PK': 216565317.0, 'PH': 108116622.0, 'QA': 2832071.0, 'RO': 19414458.0, 'RU': 145872260.0, 'SM': 34453.0, 'SG': 5804343.0, 'KR': 512